In [77]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
import os

In [3]:
df_train = pd.read_csv('../inputs/train.csv')
df_train.shape

(4576, 2)

In [183]:
train_ds = tf.data.Dataset.from_tensor_slices((df_train.Image, df_train.Class))
for x, y in train_ds.take(3):
    print(x.numpy().decode("utf-8"), y)

image9354.jpg tf.Tensor(b'Food', shape=(), dtype=string)
image9161.jpg tf.Tensor(b'Food', shape=(), dtype=string)
image557.jpg tf.Tensor(b'Attire', shape=(), dtype=string)


In [246]:
def load_preproces_image(x):
    dir_path = os.path.dirname(os.getcwd())
    image_path = os.path.join(dir_path, "dataset/Train Images", x.numpy().decode("utf-8"))
    image_raw = tf.io.read_file(image_path)
    image = tf.image.decode_image(image_raw)
    image = tf.image.resize(image, [235, 80]) # resize to max height and max width
    image = image / 255.
    return image
    
n_readers = 5

preprocessed_image_ds = train_ds.interleave(
    lambda x, y: tf.data.Dataset.from_tensors(
        (tf.py_function(image_to_array, [x], [tf.float32]), y)
    ),
    cycle_length=n_readers,
    num_parallel_calls=tf.data.AUTOTUNE
)
for x, y in preprocessed_image_ds.take(3):
    print(x.shape, y)

(1, 235, 80, 3) tf.Tensor(b'Food', shape=(), dtype=string)
(1, 235, 80, 3) tf.Tensor(b'Food', shape=(), dtype=string)
(1, 235, 80, 3) tf.Tensor(b'Attire', shape=(), dtype=string)


In [247]:
dataset = preprocessed_image_ds.shuffle(20, seed=170892).batch(32).prefetch(1)